In [ ]:
# load init moudle file

$data_import = true
load "/work/pannote/panbot_note.rb"
nil

In [ ]:
code = <<~'TASKCODE'

# save to main-db
require 'active_record'
require 'faraday'
require 'ethereum.rb'
require 'faraday'
require 'parallel'
require 'resolv-replace'

connect_str = __PARAMS__CONNECT_STR__
http_endpoint = __PARAMS__HTTP_ENDPOINT__
block_numbers = __PARAMS__BEGIN__..__PARAMS__END__

ActiveRecord::Base.establish_connection(connect_str)


## load data model from github
models = ["application_record","address","cache","block","epoch","epoch_detail","event","transfer","tx","log"]
models.each do |model| 
  url = "https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/#{model}.rb"
  body = Faraday.get(url).body
  f = open("load.rb","w")
  f.write(body)
  f.close
  load './load.rb'
end

## 数据准备


contract_name = "PancakePredictionV2"
contract_addr = "0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA".downcase
bscscan_apikey = "HIFNPQFV6MY755QMVPFIAY7F25RUTHI26Z"

client = Ethereum::HttpClient.new(http_endpoint)

api_url = "https://api.bscscan.com/api?module=contract&action=getabi&address=#{contract_addr}&apikey=#{bscscan_apikey}"
abi = JSON.parse(JSON.parse(response = Faraday.get(api_url).body)["result"])

contract = Ethereum::Contract.create(
        client: client, 
        name: contract_name, 
        address: contract_addr, 
        abi: abi)



get_abi = -> (contract_addr) {
  api_url = "https://api.bscscan.com/api?module=contract&action=getabi&address=#{contract_addr}&apikey=#{bscscan_apikey}"
  
  begin
    abi = JSON.parse(response = Faraday.get(api_url).body)["result"]

    while abi=="Max rate limit reached" do
      sleep(0.2)
      abi = JSON.parse(response = Faraday.get(api_url).body)["result"]
    end
  rescue Faraday::TimeoutError
    sleep(1)
    retry
  end
    
  return abi
}

decoder = Ethereum::Decoder.new
encoder = Ethereum::Encoder.new

Address.client = client
Address.get_abi = get_abi
Address.decoder = decoder


time = Time.now()

all_address = Tx.select(:from).distinct.all.map {|x| x.from}
addr_hash = all_address.map {|x| [x,true]}.to_h
Address.panbot_address = addr_hash

nil

puts "time #{Time.now-time} s"


def data_import_transfer(block_number,client,addr_hash)
  ## get block
  block = client.eth_get_block_by_number(block_number,true)["result"]
  block["block_number"] = block["number"].to_i(16)
  block["block_time"] = Time.at(block["timestamp"].to_i(16))

  ## get tx receipt
  transactions = block["transactions"].filter {|x| addr_hash[x["from"]] or addr_hash[x["to"]] }.map {|tx|  tx["block_number"]=block["block_number"]; tx["block_time"]=block["block_time"]; tx}

  transactions.map {|x| Address.load(x["from"]); nil }
  transactions.map {|x| Address.load(x["to"]); nil }

  transactions = Parallel.map(transactions,in_threads: 10) do |tx|
    tx["receipt"] = client.eth_get_transaction_receipt(tx["hash"])["result"]
    tx
  end

  ## process tx
  transactions.map! do |tx|
    size = tx["input"].size

    if tx["to"]!=nil then
      to = Address.find_by_addr(tx["to"])
      if to.is_contract then
        tx["method_name"],tx["method_params"] = to.decode_params(tx["input"])
      else
        tx["method_name"],tx["method_params"] = "Transfer",""
      end
    else
      tx["method_name"],tx["method_params"] = "CreateContract",""
      tx["to"] = tx["receipt"]["contractAddress"]
    end

    tx["amount"] = tx["value"].to_i(16)/1e18
    tx["block_number"] = tx["blockNumber"].to_i(16)
    tx["gas_price"] = tx["gasPrice"].to_i(16)/1e9
    tx["gas"] = tx["gas"].to_i(16)
    tx["nonce"] = tx["nonce"].to_i(16)

    tx["status"] = tx["receipt"]["status"]=="0x1" ? true : false

    tx    
  end
        
  transactions.filter {|x| x["status"]==true}.map do |tx|
    ar = Transfer.new()  
    ar.from = tx["from"]
    ar.to = tx["to"]
    ar.amount = tx["value"].to_i(16)
    ar.method_name = tx["method_name"]
      
    begin
      ar.method_params = JSON.dump(tx["method_params"])
    rescue
    end
    ar.tx_hash = tx["hash"]
    ar.block_time = tx["block_time"]
    ar.block_number = tx["blockNumber"].to_i(16)
      
    begin
      ar.save!
    rescue ActiveRecord::RecordNotUnique 
    end
  end
  
end

time = Time.now()

puts "== data import transfer - start =="
Log.log "== data import transfer - start =="



Parallel.map(block_numbers.to_a,in_threads: 10) do |block_number|  
  puts "#{Time.now} blocks: #{block_number}" if block_number%10000==0
  Log.log "#{Time.now} blocks: #{block_number}" if block_number%10000==0
  data_import_transfer(block_number,client,addr_hash)  
end

puts "== data import transfer - end =="
Log.log "== data import transfer - end =="
puts "time #{Time.now-time} s"
Log.log "time #{Time.now-time} s"

TASKCODE


"\n# save to main-db\nrequire 'active_record'\nrequire 'faraday'\nrequire 'ethereum.rb'\nrequire 'faraday'\nrequire 'parallel'\nrequire 'resolv-replace'\n\nconnect_str = __PARAMS__CONNECT_STR__\nhttp_endpoint = __PARAMS__HTTP_ENDPOINT__\nblock_numbers = __PARAMS__BEGIN__..__PARAMS__END__\n\nActiveRecord::Base.establish_connection(connect_str)\n\n\n## load data model from github\nmodels = [\"application_record\",\"address\",\"cache\",\"block\",\"epoch\",\"epoch_detail\",\"event\",\"transfer\",\"tx\",\"log\"]\nmodels.each do |model| \n  url = \"https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/\#{model}.rb\"\n  body = Faraday.get(url).body\n  f = open(\"load.rb\",\"w\")\n  f.write(body)\n  f.close\n  load './load.rb'\nend\n\n## 数据准备\n\n\ncontract_name = \"PancakePredictionV2\"\ncontract_addr = \"0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA\".downcase\nbscscan_apikey = \"HIFNPQFV6MY755QMVPFIAY7F25RUTHI26Z\"\n\nclient = Ethereum::HttpClient.new(http_endpoint)\n\napi_u

In [ ]:

db_last_transfer_block = Transfer.order(:block_number).last.block_number


last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)
db_last_block = Block.order(:block_number).last.block_number
db_last_epoch = Epoch.order(:epoch).last.epoch
last_epoch = $contract.call.current_epoch
last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)

puts "last_epoch = #{last_epoch}"
puts "db_last_epoch = #{db_last_epoch}"
puts "-----------------------"
puts "last_block = #{last_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"
puts "last_transfer_block = #{db_last_transfer_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"

puts "epochs #{last_epoch-db_last_epoch} - blocks #{last_block-db_last_block} - transfer #{db_last_block-db_last_transfer_block}"
nil


last_epoch = 45146
db_last_epoch = 45124
-----------------------
last_block = 15263226
db_last_block = 15263213
-----------------------
last_transfer_block = 15262907
db_last_block = 15263213
-----------------------
epochs 22 - blocks 13 - transfer 306


In [ ]:
begin_block = 14950000 #db_last_transfer_block
end_block = db_last_block
step = 10000

loop do
  task_end_block = [(begin_block+step)/10000*10000,begin_block+step,end_block].min
  task_name = "data-import-transfer #{begin_block} - #{task_end_block}"
  task_code = code.gsub(/__PARAMS__BEGIN__/,begin_block.to_s)
                  .gsub(/__PARAMS__END__/,task_end_block.to_s)
                  .gsub(/__PARAMS__CONNECT_STR__/,'"'+ENV["DB_CONNECT_STR"]+'"')
                  .gsub(/__PARAMS__HTTP_ENDPOINT__/,'"'+ENV["HTTP_ENDPOINT"]+'"')
  puts task_name
  Task.create_task(task_name, task_code)

  begin_block = [(begin_block+step)/10000*10000,begin_block+step].min
  break if begin_block > end_block
end



data-import-transfer 14950000 - 14960000
data-import-transfer 14960000 - 14970000
data-import-transfer 14970000 - 14980000
data-import-transfer 14980000 - 14990000
data-import-transfer 14990000 - 15000000
data-import-transfer 15000000 - 15010000
data-import-transfer 15010000 - 15020000
data-import-transfer 15020000 - 15030000
data-import-transfer 15030000 - 15040000
data-import-transfer 15040000 - 15050000
data-import-transfer 15050000 - 15060000
data-import-transfer 15060000 - 15070000
data-import-transfer 15070000 - 15080000
data-import-transfer 15080000 - 15090000
data-import-transfer 15090000 - 15100000
data-import-transfer 15100000 - 15110000
data-import-transfer 15110000 - 15120000
data-import-transfer 15120000 - 15130000
data-import-transfer 15130000 - 15140000
data-import-transfer 15140000 - 15150000
data-import-transfer 15150000 - 15160000
data-import-transfer 15160000 - 15170000
data-import-transfer 15170000 - 15180000
data-import-transfer 15180000 - 15190000
data-import-tran

In [ ]:
puts Task.all.map {|x| x.name}.sort.join("\n")

data-import-block 14958036 - 14960000
data-import-block 14960000 - 14970000
data-import-block 14970000 - 14980000
data-import-block 14980000 - 14990000
data-import-block 14990000 - 15000000
data-import-block 15000000 - 15010000
data-import-block 15010000 - 15020000
data-import-block 15020000 - 15030000
data-import-block 15030000 - 15040000
data-import-block 15040000 - 15050000
data-import-block 15050000 - 15060000
data-import-block 15060000 - 15070000
data-import-block 15070000 - 15080000
data-import-block 15080000 - 15090000
data-import-block 15090000 - 15100000
data-import-block 15100000 - 15110000
data-import-block 15110000 - 15120000
data-import-block 15120000 - 15130000
data-import-block 15130000 - 15140000
data-import-block 15140000 - 15150000
data-import-block 15150000 - 15160000
data-import-block 15160000 - 15170000
data-import-block 15170000 - 15180000
data-import-block 15180000 - 15190000
data-import-block 15190000 - 15200000
data-import-block 15200000 - 15210000
data-import-

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>